





































reboot
# PaleoClimate Plot Data

## Apply de dataset points into map


In [ ]:
import geopandas as gpd
import folium
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt


class Predication:

    def __init__(self):
        self.models_by_age = {}  # Dicionário para armazenar um modelo KNN para cada idade
        self.scalers_by_age = {}  # Dicionário para armazenar o scaler para cada idade
        self.climate_mapping = {
            'h': 1,  # Exemplo: "h" para 1 (hot)
            's': 2,  # Exemplo: "s" para 2 (semi-arid)
            't': 3   # Exemplo: "t" para 3 (temperate)
        }

    def train(self, climate_data):
        # Aplicar mapeamento aos dados de clima
        climate_data['climate'] = climate_data['climate'].map(self.climate_mapping)

        # Remover linhas com NaN na coluna 'climate'
        climate_data = climate_data.dropna(subset=['climate'])

        # Obter idades únicas
        unique_ages = climate_data['age'].unique()

        # Treinar um modelo KNN para cada idade
        for age in unique_ages:
            # Filtrar dados da idade específica
            age_data = climate_data[climate_data['age'] == age]

            # Separando features (X) e alvo (y)
            features = age_data[['lat', 'long']]
            target = age_data['climate']

            # Normalizando os dados (somente latitude e longitude)
            scaler = StandardScaler()
            features_scaled = scaler.fit_transform(features)

            # Treinar o modelo KNN (k=3) para essa idade
            knn = KNeighborsClassifier(n_neighbors=3)
            knn.fit(features_scaled, target)

            # Armazenar o modelo e o scaler para a idade
            self.models_by_age[age] = knn
            self.scalers_by_age[age] = scaler

        return climate_data

    def predict(self, points_to_predict):
        # Aplicar previsões em cada linha do CSV
        points_to_predict['predicted_climate'] = points_to_predict.apply(
            lambda row: self.prever_clima(row['lat'], row['long'], row['age']), axis=1
        )

        # Mapeando os valores numéricos de volta para categorias de clima
        inverse_climate_mapping = {v: k for k, v in self.climate_mapping.items()}
        points_to_predict['predicted_climate'] = points_to_predict['predicted_climate'].map(inverse_climate_mapping)

        return points_to_predict

    def prever_clima(self, lat, long, age):
        if age not in self.models_by_age:
            # Caso a idade não tenha sido treinada, retornar um valor padrão ou erro
            return None

        # Criar um ponto de previsão
        ponto_hipotetico = pd.DataFrame({
            'lat': [lat],
            'long': [long]
        })

        # Normalizando o ponto com o scaler específico da idade
        scaler = self.scalers_by_age[age]
        ponto_hipotetico_scaled = scaler.transform(ponto_hipotetico)

        # Fazer a previsão usando o modelo KNN para a idade específica
        knn = self.models_by_age[age]
        clima_predito = knn.predict(ponto_hipotetico_scaled)

        return clima_predito[0]


# Exemplo de uso
predicator = Predication()

# Carregar dados climáticos (substitua o caminho do arquivo com o caminho real)
climate_data = pd.read_csv('dataset/points.csv')
climate_data = predicator.train(climate_data)

# Carregar os pontos a serem previstos
points_to_predict = pd.read_csv('dataset/points_to_predict.csv')
points = predicator.predict(points_to_predict)

# Salvar o resultado em CSV
points.to_csv('dataset/predicted_points.csv', index=False)

# Exibir os dados com o estilo de visualização opcional
points.style.set_caption("Previsão de Climas para Pontos Específicos").background_gradient(cmap="coolwarm")


# Função para criar o mapa
def create_map(climate_data, points_to_predict, age_xxvmin=None, age_max=None):
    # Criar mapa com folium
    mapa = folium.Map(location=[-14, -38], zoom_start=5)

    # Função para definir ícones com base no clima
    def get_icon(climate, predicted=False):
        color = 'blue'
        icon = 'circle'

        if climate == 'h':  # Clima quente (hot)
            color = 'red'
            icon = 'info-sign' if predicted else 'circle'
        elif climate == 's':  # Clima semi-árido (semi-arid)
            color = 'blue'
            icon = 'cloud' if predicted else 'circle'
        elif climate == 't':  # Clima temperado (temperate)
            color = 'green'
            icon = 'leaf' if predicted else 'circle'

        return folium.Icon(color=color, icon=icon, prefix='glyphicon')

    # Adicionar pontos originais ao mapa
    for _, row in climate_data.iterrows():
        folium.Marker(
            location=[row['lat'], row['long']],
            popup=f"Clima Original: {row['climate']}",
            icon=get_icon(row['climate'])
        ).add_to(mapa)

    # Adicionar pontos previstos ao mapa
    for _, row in points_to_predict.iterrows():
        folium.Marker(
            location=[row['lat'], row['long']],
            popup=f"Clima Previsto: {row['predicted_climate']}",
            icon=get_icon(row['predicted_climate'], predicted=True)
        ).add_to(mapa)

    # Retornar o mapa criado
    return mapa


# Criar e exibir o mapa
mapa = create_map(climate_data, points)
#mapa.save('dataset/mapa_climatico.html')
mapa